This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [124]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'DUMMY'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [125]:
# First, import the relevant modules
import requests


In [126]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2020-03-20,end_date=2020-03020&api_key=' + API_KEY  
# into the JSON structure that will be returned
r = requests.get(url)


In [127]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.headers['Content-Type'])
print(r.text)


application/json; charset=utf-8
{"dataset_data":{"limit":null,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2020-03-20","end_date":"2020-03-20","frequency":"daily","data":[["2020-03-20",76.65,82.35,75.7,78.25,null,446534.0,34997042.95,null,null,null]],"collapse":null,"order":null}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

        QUESTION 1

In [128]:
#specify url call 
url1 = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?start_date=2017-01-01&end_date=2017-12-31&api_key=' + API_KEY  
# make get request
r1 = requests.get(url1)

QUESTION 2

In [129]:
#explicitly convert to json object
json1 = r1.json()
#convert to dictionary
dict_zeiss_2017 = dict(json1["dataset_data"])
#check dictionary keys and date key-values
print(dict_zeiss_2017.keys())
print('start date =' + dict_zeiss_2017['start_date'])
print('end date =' + dict_zeiss_2017['end_date'])

dict_keys(['limit', 'transform', 'column_index', 'column_names', 'start_date', 'end_date', 'frequency', 'data', 'collapse', 'order'])
start date =2017-01-01
end date =2017-12-31


After inspecting dictionary, it is a mess to work with so going to manually wrangle into more intuitive shape given instructions not to use pandas or json libraries

In [130]:
#Create lists for main data (chosing to ignore metadata )
n = len(dict_zeiss_2017['data'])                                    #number of days / data in 2017

dates = [dict_zeiss_2017['data'][j][0] for j in range(n)]           #list comprehension for ordered dates
opening = [dict_zeiss_2017['data'][j][1] for j in range(n)]         #list comprehension for opening price in same order as dates
high = [dict_zeiss_2017['data'][j][2] for j in range(n)]            #list comprehension for high price in same order as dates
low = [dict_zeiss_2017['data'][j][3] for j in range(n)]             #list comprehension for low price in same order as dates
closing = [dict_zeiss_2017['data'][j][4] for j in range(n)]         #list comprehension for closing price in same order as dates
volume = [dict_zeiss_2017['data'][j][6] for j in range(n)]          #list comprehension for volume in same order as dates
turnover = [dict_zeiss_2017['data'][j][7] for j in range(n)]        #list comprehension for turnover in same order as dates

#generate clean dictionary of data only using dictionary comprtehension and lists
clean_zeiss_2017 = { dates[j]:{'open': opening[j], 'high': high[j], 'low': low[j], 'close':closing [j], 'volume': volume[j],
                     'turnover': turnover[j]} for j in range(n) }

In [131]:
#check dictionary for sample date
clean_zeiss_2017['2017-12-29']

{'open': 51.76,
 'high': 51.94,
 'low': 51.45,
 'close': 51.76,
 'volume': 34640.0,
 'turnover': 1792304.0}

QUESTION 3

In [132]:
#remove NoneType
clean_opening = [price for price in opening if price is not None]
#calculate min of list
open_low = min(clean_opening)
open_high = max(clean_opening)

print('open_low = ' + str(open_low) )
print('open_high = ' + str(open_high) )

open_low = 34.0
open_high = 53.11


QUESTION 4

In [133]:
#Generate Daily High - Low since this field was None in original json (no None data in high or low)
daily_range = [high[j] - low[j] for j in range(n)]
max_change = max(daily_range) 
print('max_change = ' + str(max_change) )

max_change = 2.8100000000000023


QUESTION 5

In [134]:
#Generate day on day change (closing has no None data) 
dod_delta= [closing[j+1] - closing[j] for j in range(n-1)]
max_dod_delta = max(dod_delta)
print('max_dod_delta = ' + str(max_dod_delta) )

max_dod_delta = 2.559999999999995


QUESTION 6

In [135]:
#Calculate average volume using sum / count formula
ave_volume = sum(volume) / len(volume)
print('ave_volume = ' + str(ave_volume) )

ave_volume = 89124.33725490196


QUESTION 7

In [137]:
volume_sort = sorted(volume)          #Sort volume in ascending order
num = len(volume)                     # count of volume

#formuala for calculating median 

if num //2 !=0:                                                  #if count is odd the middle element given by count//2+1
    median = volume_sort[num//2 + 1]
else:
    median = (volume_sort[num/2] + volume_sort[num/2 + 1]) / 2   #if not odd then given by sum of "middle two elements" 

print('median_volume = ' + str(median))
   

median_volume = 76600.0
